# Полносвязные нейронные сети

В предыдущем домашнем задании вы реализовали полносвязную двухслойную нейронную сеть на датасете CIFAR-10. Реализация была простой, но не очень эффективной, поскольку не использовала модульные конструкции для вычисления потерь и градиента. Это работает для простых сетей, но будет непрактично для масштабных моделей. В идеале мы хотим построить сеть с использованием модульной конструкции, чтобы мы могли реализовать разные типы слоёв изолированно, а затем объединять их в модели с разными архитектурами.

В этом упражнении мы создадим полносвязные сети, используя модульный подход. Для каждого слоя мы напишем функции `forward` и` backward`. Функция `forward` будет получать входные данные, веса и другие параметры и возвращать как выходные данные, так и объект `cache`, хранящий необходимые для обратного прохода сведения, например:

```python
def layer_forward(x, w):
  """ Получаем исходные данные x и веса w """
  # Делаем некотоорые вычисления ...
  z = # ... Промежуточное значение
  # Делаем ещё немного вычислений ...
  out = # Выход
   
  cache = (x, w, z, out) # Значения, необходимые для вычисления градиента
   
  return out, cache
```

Обратный проход имеет на входе восходящие производные и объект `cache` и вовзвращает градиенты относительно входных данных и весов, например:

```python
def layer_backward(dout, cache):
  """
  Принимаем dout (производная потерь по отношению к выходным данным) и cache,
  вычисляем производные по отношению к исходным данным.
  """
  # Распаковываем значения кэша
  x, w, z, out = cache
  
  # Используем их для вычисления производных
  dx = # Производная потерь по отношению к x
  dw = # Производная потерь по отношению к w
  
  return dx, dw
```

После реализации нескольких таких слоёв мы сможем легко объединить их для создания классификаторов с различными архитектурами.

Помимо реализации полносвязных сетей произвольной глубины мы также изучим различные правила обновления для оптимизации, а также попробуем применить Dropout в качестве регуляризатора. Для более эффективной оптимизации глубоких сетей мы будем использовать пакетную нормализацию и нормализацию слоёв.
  

In [ ]:
# Как обычно, немного настроек
from __future__ import print_function
import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# для автоматической загрузки внешних модулей
# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ возвращает относительную ошибку """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# Загружаем (предварительно обработанный) датасет CIFAR10.

data = get_CIFAR10_data()
for k, v in list(data.items()):
  print(('%s: ' % k, v.shape))

# Аффинный слой: foward
Откройте файл `cs231n/layers.py` и реализуйте функцию `affine_forward`.

Когда вы закончите, вы можете проверить свою реализацию, выполнив следующее:

In [ ]:
# Тестируем функцию affine_forward

num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)

out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])

# Сравните ваши результаты с нашими. Ошибка должна быть около e-9 или меньше.
print('Testing affine_forward function:')
print('difference: ', rel_error(out, correct_out))

# Аффинный слой: backward
Теперь реализуйте функцию `affine_backward` и протестируйте её, используя численную градиентную проверку.

In [ ]:
# Тестируем функцию affine_backward
np.random.seed(231)
x = np.random.randn(10, 2, 3)
w = np.random.randn(6, 5)
b = np.random.randn(5)
dout = np.random.randn(10, 5)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

# Ошибка должна быть около e-10 или меньше
print('Testing affine_backward function:')
print('dx error: ', rel_error(dx_num, dx))
print('dw error: ', rel_error(dw_num, dw))
print('db error: ', rel_error(db_num, db))

# ReLU активация: forward
Напишите прямой проход для активации ReLU в функции `relu_forward` и протестируйте свою реализацию:

In [ ]:
# Тестируем функцию relu_forward

x = np.linspace(-0.5, 0.5, num=12).reshape(3, 4)

out, _ = relu_forward(x)
correct_out = np.array([[ 0.,          0.,          0.,          0.,        ],
                        [ 0.,          0.,          0.04545455,  0.13636364,],
                        [ 0.22727273,  0.31818182,  0.40909091,  0.5,       ]])

# Ошибка должна быть порядка e-8
print('Testing relu_forward function:')
print('difference: ', rel_error(out, correct_out))

# ReLU активация: backward
Теперь реализуйте обратный проход для ReLU в функции `relu_backward` и протестируйте свою реализацию, используя численный градиент:

In [ ]:
np.random.seed(231)
x = np.random.randn(10, 10)
dout = np.random.randn(*x.shape)

dx_num = eval_numerical_gradient_array(lambda x: relu_forward(x)[0], x, dout)

_, cache = relu_forward(x)
dx = relu_backward(dout, cache)

# Ошибка должна быть порядка e-12
print('Testing relu_backward function:')
print('dx error: ', rel_error(dx_num, dx))

# "Сэндвич-слои"
Существует несколько общих моделей слоёв, которые часто используются в нейронных сетях. Например, за аффинными слоями часто следует нелинейность ReLU. Чтобы упростить эти шаблоны, мы определим несколько вспомогательных слоёв в файле `cs231n/layer_utils.py`.

А пока взгляните на функции `affine_relu_forward` и` affine_relu_backward` и выполните следующий код для проверки обратного прохода с помощью числового градиента:

In [ ]:
from cs231n.layer_utils import affine_relu_forward, affine_relu_backward
np.random.seed(231)
x = np.random.randn(2, 3, 4)
w = np.random.randn(12, 10)
b = np.random.randn(10)
dout = np.random.randn(2, 10)

out, cache = affine_relu_forward(x, w, b)
dx, dw, db = affine_relu_backward(dout, cache)

dx_num = eval_numerical_gradient_array(lambda x: affine_relu_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_relu_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_relu_forward(x, w, b)[0], b, dout)

# Относительная ошибка должна быть около e-10 или меньше
print('Testing affine_relu_forward and affine_relu_backward:')
print('dx error: ', rel_error(dx_num, dx))
print('dw error: ', rel_error(dw_num, dw))
print('db error: ', rel_error(db_num, db))

# Слои потерь: Softmax и SVM
Вы реализовали эти функции потерь в предыдущем домашнем задании, поэтому мы просто напишем их здесь. Вы всё равно должны убедиться, что понимаете, как они работают, посмотрев их реализации в `cs231n/Layers.py`.

Можете убедиться, что они работают правильно, запустив следующее:

In [ ]:
np.random.seed(231)
num_classes, num_inputs = 10, 50
x = 0.001 * np.random.randn(num_inputs, num_classes)
y = np.random.randint(num_classes, size=num_inputs)

dx_num = eval_numerical_gradient(lambda x: svm_loss(x, y)[0], x, verbose=False)
loss, dx = svm_loss(x, y)

# Тестируем функцию svm_loss. Потери должны быть около 9, 
# а ошибка dx должна быть порядка e-9
print('Testing svm_loss:')
print('loss: ', loss)
print('dx error: ', rel_error(dx_num, dx))

dx_num = eval_numerical_gradient(lambda x: softmax_loss(x, y)[0], x, verbose=False)
loss, dx = softmax_loss(x, y)

# Тестируем функцию softmax_loss. Потери должны быть близки к 2.3
# а ошибка dx должна быть около e-8
print('\nTesting softmax_loss:')
print('loss: ', loss)
print('dx error: ', rel_error(dx_num, dx))

# Двухслойная сеть
В предыдущем задании вы создали двухслойную нейронную сеть в одном классе. Теперь, когда вы реализовали модульные версии необходимых слоёв, вы переопределите эту двухслойную сеть с помощью модульных реализаций.

Откройте файл `cs231n/classifiers/fc_net.py` и завершите реализацию класса` TwoLayerNet`. Этот класс будет служить основой для других сетей, которые вы будете реализовывать в этом задании, поэтому убедитесь, что вы понимаете, как он работает. Можете запустить ячейку ниже, чтобы проверить свою реализацию.

In [ ]:
np.random.seed(231)
N, D, H, C = 3, 5, 50, 7
X = np.random.randn(N, D)
y = np.random.randint(C, size=N)

std = 1e-3
model = TwoLayerNet(input_dim=D, hidden_dim=H, num_classes=C, weight_scale=std)

print('Testing initialization ... ')
W1_std = abs(model.params['W1'].std() - std)
b1 = model.params['b1']
W2_std = abs(model.params['W2'].std() - std)
b2 = model.params['b2']
assert W1_std < std / 10, 'First layer weights do not seem right'
assert np.all(b1 == 0), 'First layer biases do not seem right'
assert W2_std < std / 10, 'Second layer weights do not seem right'
assert np.all(b2 == 0), 'Second layer biases do not seem right'

print('Testing test-time forward pass ... ')
model.params['W1'] = np.linspace(-0.7, 0.3, num=D*H).reshape(D, H)
model.params['b1'] = np.linspace(-0.1, 0.9, num=H)
model.params['W2'] = np.linspace(-0.3, 0.4, num=H*C).reshape(H, C)
model.params['b2'] = np.linspace(-0.9, 0.1, num=C)
X = np.linspace(-5.5, 4.5, num=N*D).reshape(D, N).T
scores = model.loss(X)
correct_scores = np.asarray(
  [[11.53165108,  12.2917344,   13.05181771,  13.81190102,  14.57198434, 15.33206765,  16.09215096],
   [12.05769098,  12.74614105,  13.43459113,  14.1230412,   14.81149128, 15.49994135,  16.18839143],
   [12.58373087,  13.20054771,  13.81736455,  14.43418138,  15.05099822, 15.66781506,  16.2846319 ]])
scores_diff = np.abs(scores - correct_scores).sum()
assert scores_diff < 1e-6, 'Problem with test-time forward pass'

print('Testing training loss (no regularization)')
y = np.asarray([0, 5, 1])
loss, grads = model.loss(X, y)
correct_loss = 3.4702243556
assert abs(loss - correct_loss) < 1e-10, 'Problem with training-time loss'

model.reg = 1.0
loss, grads = model.loss(X, y)
correct_loss = 26.5948426952
assert abs(loss - correct_loss) < 1e-10, 'Problem with regularization loss'

# Ошибки должны быть около e-7 или меньше
for reg in [0.0, 0.7]:
  print('Running numeric gradient check with reg = ', reg)
  model.reg = reg
  loss, grads = model.loss(X, y)

  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False)
    print('%s relative error: %.2e' % (name, rel_error(grad_num, grads[name])))

# Solver (решатель)
В предыдущем задании логика обучения моделей была связана с самими моделями. Следуя модульной конструкции, для этого задания мы отделим логику обучения моделей в отдельный класс.

Откройте файл `cs231n/solver.py` и просмотрите его, чтобы ознакомиться с API. После этого используйте экземпляр `Solver` для обучения `TwoLayerNet`, который должен достигнуть по меньшей мере `50%` точности на оценочном наборе.

In [ ]:
model = TwoLayerNet()
solver = None

##############################################################################
# TODO: Используйте экземпляр Solver для обучения TwoLayerNet, который       #
# должен достигнуть минимум 50% точности на оценочном наборе данных.         #
##############################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

In [ ]:
# Запустите эту ячейку, чтобы визуализировать потери и точность обучения

plt.subplot(2, 1, 1)
plt.title('Training loss')
plt.plot(solver.loss_history, 'o')
plt.xlabel('Iteration')

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(solver.train_acc_history, '-o', label='train')
plt.plot(solver.val_acc_history, '-o', label='val')
plt.plot([0.5] * len(solver.val_acc_history), 'k--')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()

# Многослойная сеть
Далее вы создадите полносвязную сеть с произвольным количеством скрытых слоёв.

Ознакомьтесь с классом `FullyConnectedNet` в файле` cs231n/classifiers/fc_net.py`.

Реализуйте инициализацию, прямой проход и обратный проход. На данный момент не беспокойтесь о реализации dropout или нормализации; мы добавим эти функции в ближайшее время.

## Начальные потери и градиентная проверка

В качестве проверки работоспособности вашего кода выполните следующую ячейку, чтобы оценить первоначальные потери и провести градиентную проверку нейросети как с регуляризацией, так и без нее. 

Для градиентной проверки следует ожидать ошибки около 1e-7 или менее.

In [ ]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

for reg in [0, 3.14]:
  print('Running check with reg = ', reg)
  model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            reg=reg, weight_scale=5e-2, dtype=np.float64)

  loss, grads = model.loss(X, y)
  print('Initial loss: ', loss)
  
  # Большинство ошибок должно быть порядка e-7 или меньше.   
  # ПРИМЕЧАНИЕ: Вполне возможно получить ошибку порядка e-5 для W2
  # проверка при reg = 0.0
  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
    print('%s relative error: %.2e' % (name, rel_error(grad_num, grads[name])))

В качестве еще одной проверки работоспособности убедитесь, что вы можете переобучить сеть на небольшом наборе данных из 50 изображений. Сначала попробуем использовать трёхслойную сеть со 100 единицами в каждом скрытом слое. В следующей ячейке настройте **learning rate** и **weight initialization scale**, чтобы переобучить сеть и достичь 100% точности в течение 20 эпох.

In [ ]:
# TODO: Используйте трёхслойную нейросеть, чтобы переобучить её на 50
# учебных примерах, настраивая только скорость обучения и шкалу инициализации.

num_train = 50
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 1e-2   # Экспериментируйте с этим!
learning_rate = 1e-4  # Экспериментируйте с этим!
model = FullyConnectedNet([100, 100],
              weight_scale=weight_scale, dtype=np.float64)
solver = Solver(model, small_data,
                print_every=10, num_epochs=20, batch_size=25,
                update_rule='sgd',
                optim_config={
                  'learning_rate': learning_rate,
                }
         )
solver.train()

plt.plot(solver.loss_history, 'o')
plt.title('Training loss history')
plt.xlabel('Iteration')
plt.ylabel('Training loss')
plt.show()

Теперь попробуйте использовать пятислойную сеть со 100 единицами на каждом слое, чтобы переобучить её на 50 образцах. Настройки остаются те же, и вы должны достичь 100% точности обучения в течение 20 эпох.

In [ ]:
# TODO: Используйте пятислойную нейросеть, чтобы переобучить её на 50
# учебных примерах, настраивая только скорость обучения и шкалу инициализации.

num_train = 50
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

learning_rate = 2e-3  # Экспериментируйте с этим!
weight_scale = 1e-5   # Экспериментируйте с этим!
model = FullyConnectedNet([100, 100, 100, 100],
                weight_scale=weight_scale, dtype=np.float64)
solver = Solver(model, small_data,
                print_every=10, num_epochs=20, batch_size=25,
                update_rule='sgd',
                optim_config={
                  'learning_rate': learning_rate,
                }
         )
solver.train()

plt.plot(solver.loss_history, 'o')
plt.title('Training loss history')
plt.xlabel('Iteration')
plt.ylabel('Training loss')
plt.show()

# Правила обновления
До сих пор мы использовали обычный стохастический градиентный спуск (SGD) в качестве правила обновления. Более сложные правила могут упростить обучение для глубоких нейросетей. Мы реализуем несколько наиболее часто используемых правил обновления и сравним их с обычным SGD.

# SGD + Импульс
Стохастический градиентный спуск с импульсом является широко используемым правилом обновления, которое заставляет глубокие сети сходиться быстрее, чем при обычном градиентном спуске.

Откройте файл `cs231n/optim.py` и прочитайте документацию, чтобы разобраться с API. Реализуйте правило обновления SGD+momentum в функции `sgd_momentum` и запустите ячейку ниже, чтобы проверить свою реализацию. Ошибка должна быть меньше e-8.

In [ ]:
from cs231n.optim import sgd_momentum

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
v = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-3, 'velocity': v}
next_w, _ = sgd_momentum(w, dw, config=config)

expected_next_w = np.asarray([
  [ 0.1406,      0.20738947,  0.27417895,  0.34096842,  0.40775789],
  [ 0.47454737,  0.54133684,  0.60812632,  0.67491579,  0.74170526],
  [ 0.80849474,  0.87528421,  0.94207368,  1.00886316,  1.07565263],
  [ 1.14244211,  1.20923158,  1.27602105,  1.34281053,  1.4096    ]])
expected_velocity = np.asarray([
  [ 0.5406,      0.55475789,  0.56891579, 0.58307368,  0.59723158],
  [ 0.61138947,  0.62554737,  0.63970526,  0.65386316,  0.66802105],
  [ 0.68217895,  0.69633684,  0.71049474,  0.72465263,  0.73881053],
  [ 0.75296842,  0.76712632,  0.78128421,  0.79544211,  0.8096    ]])

# Относительная ошибка должна быть e-8 или меньше
print('next_w error: ', rel_error(next_w, expected_next_w))
print('velocity error: ', rel_error(expected_velocity, config['velocity']))

Как только вы выполните код выше, запустите следующую ячейку, чтобы обучить шестислойную сеть с двумя правилами: SGD и SGD + импульс. Вы должны увидеть, что SGD + импульс сходится быстрее.

In [ ]:
num_train = 4000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

solvers = {}

for update_rule in ['sgd', 'sgd_momentum']:
  print('running with ', update_rule)
  model = FullyConnectedNet([100, 100, 100, 100, 100], weight_scale=5e-2)

  solver = Solver(model, small_data,
                  num_epochs=5, batch_size=100,
                  update_rule=update_rule,
                  optim_config={
                    'learning_rate': 5e-3,
                  },
                  verbose=True)
  solvers[update_rule] = solver
  solver.train()
  print()

plt.subplot(3, 1, 1)
plt.title('Training loss')
plt.xlabel('Iteration')

plt.subplot(3, 1, 2)
plt.title('Training accuracy')
plt.xlabel('Epoch')

plt.subplot(3, 1, 3)
plt.title('Validation accuracy')
plt.xlabel('Epoch')

for update_rule, solver in solvers.items():
  plt.subplot(3, 1, 1)
  plt.plot(solver.loss_history, 'o', label="loss_%s" % update_rule)
  
  plt.subplot(3, 1, 2)
  plt.plot(solver.train_acc_history, '-o', label="train_acc_%s" % update_rule)

  plt.subplot(3, 1, 3)
  plt.plot(solver.val_acc_history, '-o', label="val_acc_%s" % update_rule)
  
for i in [1, 2, 3]:
  plt.subplot(3, 1, i)
  plt.legend(loc='upper center', ncol=4)
plt.gcf().set_size_inches(15, 15)
plt.show()

# RMSProp и Adam
RMSProp и Адам - это правила обновления, которые устанавливают скорости обучения для каждого параметра, используя скользящее среднее.

В файле`cs231n/optim.py` реализуйте RMSProp в функции `rmsprop` и Adam в функции `adam`, а затем проверьте свой код с помощью тестов ниже.

In [ ]:
# Тестируем реализацию RMSProp
from cs231n.optim import rmsprop

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
cache = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-2, 'cache': cache}
next_w, _ = rmsprop(w, dw, config=config)

expected_next_w = np.asarray([
  [-0.39223849, -0.34037513, -0.28849239, -0.23659121, -0.18467247],
  [-0.132737,   -0.08078555, -0.02881884,  0.02316247,  0.07515774],
  [ 0.12716641,  0.17918792,  0.23122175,  0.28326742,  0.33532447],
  [ 0.38739248,  0.43947102,  0.49155973,  0.54365823,  0.59576619]])
expected_cache = np.asarray([
  [ 0.5976,      0.6126277,   0.6277108,   0.64284931,  0.65804321],
  [ 0.67329252,  0.68859723,  0.70395734,  0.71937285,  0.73484377],
  [ 0.75037008,  0.7659518,   0.78158892,  0.79728144,  0.81302936],
  [ 0.82883269,  0.84469141,  0.86060554,  0.87657507,  0.8926    ]])

# Ошибка должна быть e-7 или меньше
print('next_w error: ', rel_error(expected_next_w, next_w))
print('cache error: ', rel_error(expected_cache, config['cache']))

In [ ]:
# Тестируем реализацию Adam
from cs231n.optim import adam

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
m = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)
v = np.linspace(0.7, 0.5, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-2, 'm': m, 'v': v, 't': 5}
next_w, _ = adam(w, dw, config=config)

expected_next_w = np.asarray([
  [-0.40094747, -0.34836187, -0.29577703, -0.24319299, -0.19060977],
  [-0.1380274,  -0.08544591, -0.03286534,  0.01971428,  0.0722929],
  [ 0.1248705,   0.17744702,  0.23002243,  0.28259667,  0.33516969],
  [ 0.38774145,  0.44031188,  0.49288093,  0.54544852,  0.59801459]])
expected_v = np.asarray([
  [ 0.69966,     0.68908382,  0.67851319,  0.66794809,  0.65738853,],
  [ 0.64683452,  0.63628604,  0.6257431,   0.61520571,  0.60467385,],
  [ 0.59414753,  0.58362676,  0.57311152,  0.56260183,  0.55209767,],
  [ 0.54159906,  0.53110598,  0.52061845,  0.51013645,  0.49966,   ]])
expected_m = np.asarray([
  [ 0.48,        0.49947368,  0.51894737,  0.53842105,  0.55789474],
  [ 0.57736842,  0.59684211,  0.61631579,  0.63578947,  0.65526316],
  [ 0.67473684,  0.69421053,  0.71368421,  0.73315789,  0.75263158],
  [ 0.77210526,  0.79157895,  0.81105263,  0.83052632,  0.85      ]])

# Ошибка должна быть e-7 или меньше
print('next_w error: ', rel_error(expected_next_w, next_w))
print('v error: ', rel_error(expected_v, config['v']))
print('m error: ', rel_error(expected_m, config['m']))

После отладки ваших реализаций RMSProp и Adam выполните следующий код, чтобы обучить две глубокие сети, используя новые правила обновления:

In [ ]:
learning_rates = {'rmsprop': 1e-4, 'adam': 1e-3}
for update_rule in ['adam', 'rmsprop']:
  print('running with ', update_rule)
  model = FullyConnectedNet([100, 100, 100, 100, 100], weight_scale=5e-2)

  solver = Solver(model, small_data,
                  num_epochs=5, batch_size=100,
                  update_rule=update_rule,
                  optim_config={
                    'learning_rate': learning_rates[update_rule]
                  },
                  verbose=True)
  solvers[update_rule] = solver
  solver.train()
  print()

plt.subplot(3, 1, 1)
plt.title('Training loss')
plt.xlabel('Iteration')

plt.subplot(3, 1, 2)
plt.title('Training accuracy')
plt.xlabel('Epoch')

plt.subplot(3, 1, 3)
plt.title('Validation accuracy')
plt.xlabel('Epoch')

for update_rule, solver in list(solvers.items()):
  plt.subplot(3, 1, 1)
  plt.plot(solver.loss_history, 'o', label=update_rule)
  
  plt.subplot(3, 1, 2)
  plt.plot(solver.train_acc_history, '-o', label=update_rule)

  plt.subplot(3, 1, 3)
  plt.plot(solver.val_acc_history, '-o', label=update_rule)
  
for i in [1, 2, 3]:
  plt.subplot(3, 1, i)
  plt.legend(loc='upper center', ncol=4)
plt.gcf().set_size_inches(15, 15)
plt.show()

# Обучите хорошую модель!
Обучите наилучшую полносвязную модель с помощью датасета CIFAR-10 и сохраните её в переменную `best_model`. Вы должны получить по крайней мере 50% точности на проверочном наборе.

Для достижения наилучших результатов вы можете сначала попробовать выполнить задания в файлах `BatchNormalization.ipynb` и `Dropout.ipynb`.

In [ ]:
best_model = None
################################################################################
# TODO: Обучить наилучшую полносвязную модель на датасете CIFAR-10.            #                                                   #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Проверьте свою модель!
Запустите свою лучшую модель на проверочном и тестовом наборах данных. Вы должны достичь не менее 50% точности на проверочном наборе.

In [ ]:
y_test_pred = np.argmax(best_model.loss(data['X_test']), axis=1)
y_val_pred = np.argmax(best_model.loss(data['X_val']), axis=1)
print('Validation set accuracy: ', (y_val_pred == data['y_val']).mean())
print('Test set accuracy: ', (y_test_pred == data['y_test']).mean())